In [9]:
# TO EXPORT TO HTML:
# jupyter nbconvert <notebook name>.ipynb --to html
# flags:
    # --no-input = exclude all code from showing

# load data
import pandas as pd
import geopandas as gpd
import plotly.express as px
import contextily as cx


#---------load data-----------
# sharing URL creating through gdrive interface
sharing_url = 'https://drive.google.com/file/d/1NB326BZFDwJ6xIeRL4MiW-Fwp8BHT-Vk/view?usp=sharing'
file_id = sharing_url.split('/')[-2]

# create direct link
# create direct link with https://sites.google.com/site/gdocs2direct/
direct_url = fr'https://drive.google.com/uc?export=download&id={file_id}'

# TEST = r'~/projects/land_tax_analysis_data/SACRAMENTO.parquet'
use_cols = ['SITUS_NUMBER', 'SITUS_CITY', 'SITUS_STREET', 'ZIP', 'LU_GENERAL', 'LU_SPECIF',
          'LANDVAL', 'IMP_VAL', 'PROPTAX', 'LANDTAX', 'TAXDIF']

# NOTE - geopandas cannot read non-local parquet file. Must first load to pandas then convert to geodf
gdf = pd.read_parquet(direct_url, columns=[*use_cols, 'geometry']) \
    .rename(columns={'geometry': 'geom_wkb'})
gdf = gpd.GeoDataFrame(gdf, geometry=gpd.GeoSeries.from_wkb(gdf['geom_wkb']))
print(f"{gdf.shape[0]:,.0f} parcels loaded.");

#---------compute land tax rate that corresponds to revenue neutrality vs. property tax-------
gdf['landvpct'] = gdf['LANDVAL'] / (gdf['LANDVAL'] + gdf['IMP_VAL'])
gdf['taxpctdif'] = gdf['TAXDIF'] / gdf['PROPTAX']

ltaxrate = gdf['LANDTAX'].mean() / gdf['LANDVAL'].mean()
ltaxrate = f'{ltaxrate*100:.3f}%'
print("Base assumed PROPERTY TAX RATE: 1.00%")
print(f"LAND TAX RATE needed for revenue neutrality: {ltaxrate}")

229,183 parcels loaded.
Base assumed PROPERTY TAX RATE: 1.00%
LAND TAX RATE needed for revenue neutrality: 3.549%


In [6]:
# histogram for all land use types showing what the change would be
q_hi = gdf['TAXDIF'].quantile(0.975) # upper limit
q_lo = gdf['TAXDIF'].quantile(0.025) # lower limit
hist_df = gdf.loc[(gdf['TAXDIF'] < q_hi) & (gdf['TAXDIF'] > q_lo)]

fig = px.histogram(hist_df, x='TAXDIF', nbins=20)
fig.update_layout(bargap=0.2)
fig.show()

In [9]:
# What % of parcels see decrease? Especially for residential parcels?
gdfr = gdf[gdf['LU_GENERAL'] == 'Residential']
gdfnr = gdf[gdf['LU_GENERAL'] != 'Residential']

# what % of res parcels would see smaller prop tax?
pct_res_red = gdfr[gdfr['TAXDIF'] <= 0].shape[0] / gdfr.shape[0]
print(f"{pct_res_red*100:.0f}% of residential parcels would see decrease")

pct_nres_red = gdfnr[gdfnr['TAXDIF'] <= 0].shape[0] / gdfnr.shape[0]
print(f"{pct_nres_red*100:.0f}% of non-residential parcels would see decrease")

56% of residential parcels would see decrease
23% of non-residential parcels would see decrease


In [10]:
# median change by land use type, and for different residential types
median_lugen = gdf.groupby('LU_GENERAL')['TAXDIF'].median() \
            .reset_index() \
            .sort_values(by='TAXDIF') \
            .rename(columns={'TAXDIF': 'med_TAXDIF'})
cnt_x_lugen = gdf.groupby('LU_GENERAL')['TAXDIF'].count().reset_index().rename(columns={'TAXDIF':'pcl_count'})
display(median_lugen.merge(cnt_x_lugen, on='LU_GENERAL').sort_values(by='med_TAXDIF', ascending=False))

# split out by type of residential
median_x_res = gdfr.groupby('LU_SPECIF')['TAXDIF'].median() \
            .reset_index() \
            .sort_values(by='TAXDIF') \
            .rename(columns={'TAXDIF': 'med_TAXDIF'})
cnt_x_res = gdfr.groupby('LU_SPECIF')['TAXDIF'].count().reset_index().rename(columns={'TAXDIF':'pcl_count'})
summarytbl = median_x_res.merge(cnt_x_res, on='LU_SPECIF')
summarytbl
# fig = px.bar(median_x_res, y="LU_SPECIF", x="med_TAXDIF", orientation='h')
# fig.show()

,LU_GENERAL,med_TAXDIF,pcl_count
10,Agricultural,4061.238024,237
9,Retail / Commercial,2187.785449,3973
8,Vacant,2132.902044,6517
7,Recreational,698.058747,145
6,Public / Utilities,436.079576,9
5,Office,379.349190,2073
4,Miscellaneous,3.186784,1144
3,Residential,-110.543469,211223
2,Industrial,-241.122698,2861
1,Church / Welfare,-256.060704,740


,LU_SPECIF,med_TAXDIF,pcl_count
0,High Rise Apartment,-144186.978178,64
1,Hotel,-38449.261551,61
2,Low Rise Apartment,-2633.751082,2266
3,Bed & Breakfast,-2222.031853,6
4,Motel,-2034.379866,65
5,Court,-603.818940,176
6,Boarding House,-427.141358,6
7,Single Family,-112.457417,197434
8,Rooming House,-89.989963,18
9,Common Area,0.254943,34


In [3]:
# Map out as interactive, explorable map

zips = [95816, 95817]
df_to_map = gdf.loc[gdf['ZIP'].isin(zips)]

# gdf['SITUS_STREET'] = gdf['SITUS_STREET'].fillna('')
# df_to_map = gdf.loc[gdf['SITUS_STREET'].str.contains('FREEPORT')]

# break points for color categories; make universal for entire dataframe, not just selection shown
pct_bins = [-0.5, -0.1, -0.05, 0.05, 0.1, 0.5, gdf['taxpctdif'].max()] 

m = df_to_map.explore(
    column="taxpctdif",  # make choropleth based on "POP2010" column
    scheme='user_defined', 
    classification_kwds={'bins':pct_bins},
    cmap='coolwarm', # color map from blues to reds
    legend=True,  # show legend
    tooltip=False,  # hide tooltip
    popup=[*use_cols, 'taxpctdif'],  # show popup (on-click)
    colorbar=False, # use legend with bin values rather than auto-generated color ramps
    name="tax difference",  # name of the layer in the map
    style_kwds=dict(weight=0) # don't have borders on parcel polygon icons
)

m

In [72]:
# Option to map as static map, maybe best when mapping a lot
# https://geopandas.org/en/stable/docs/user_guide/mapping.html
# https://geopandas.org/en/stable/gallery/plotting_basemap_background.html
import contextily as cx


df_to_map = gdf
# zips = [95811, 95814, 95816, 95817, 95820, 95822, 95824]
# df_to_map = gdf.loc[gdf['ZIP'].isin(zips)]

# ax = df_to_map.plot(
#     column="taxpctdif",
#     legend=True,
#     cmap='coolwarm',
#     figsize=(20, 20),
#     scheme="User_Defined", 
#     classification_kwds=dict(bins=pct_bins)
# )

# cx.add_basemap(ax, source=cx.providers.CartoDB.Positron)

In [ ]:
# Make map, but do as change in dollar amount, not percent